### Initialize Spark session

In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path

println("Initializing Spark context...")
val conf = new SparkConf().setAppName("Example App")
val spark: SparkSession = SparkSession.builder.config(conf).getOrCreate()
//if you look in Spark Master UI, a application will be running after this

Initializing Spark context...


conf = org.apache.spark.SparkConf@7cff3ed5
spark = org.apache.spark.sql.SparkSession@45243158


org.apache.spark.sql.SparkSession@45243158

### Custom Jar import

In [29]:
println("Import custom jar in the notebook with a special Toree directive")
%AddJar file:///app/app.jar

Import custom jar in the notebook with a special Toree directive
Starting download from file:///app/app.jar
Finished download of app.jar


In [30]:
println("Importing custom class")
import app.Point
val p = new Point(1,2)
p.move(3,5)

Importing custom class
Point x location : 4
Point y location : 7


p = app.Point@6db4b5e6


app.Point@6db4b5e6

### Hello World using remote Spark master

In [2]:
println("************")
println("Hello, world!")
val rdd = spark.sparkContext.parallelize(Array(1 to 10))
rdd.count()
println("************")

************
Hello, world!
************


rdd = ParallelCollectionRDD[0] at parallelize at <console>:35


ParallelCollectionRDD[0] at parallelize at <console>:35

### HDFS tests

In [3]:
val hdfsPrefix = sys.env("HDFS_URL")
val hadoopConf = new Configuration()
hadoopConf.set("fs.defaultFS", sys.env("HDFS_URL"))
val hdfs = FileSystem.get(hadoopConf)

hdfsPrefix = hdfs://namenode1:8020
hadoopConf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
hdfs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_1315836187_43, ugi=root (auth:SIMPLE)]]


DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_1315836187_43, ugi=root (auth:SIMPLE)]]

In [4]:
println("Copying sample file to HDFS...")
val srcPath = new Path("/notebooks/people.csv")
val destPath = new Path("hdfs:///test/people.csv")
hdfs.copyFromLocalFile(srcPath, destPath)

Copying sample file to HDFS...


srcPath = /notebooks/people.csv
destPath = hdfs:/test/people.csv


hdfs:/test/people.csv

In [5]:
println("Load CSV from HDFS to Dataframe")
val df = spark.read
          .format("com.databricks.spark.csv")
          .option("inferSchema", "true")
          .option("header", "true")
          .load(hdfsPrefix + "/test/people.csv")
df.describe().show()

Load CSV from HDFS to Dataframe
+-------+-----------------+------------------+------------------+
|summary|            Index|            Height|            Weight|
+-------+-----------------+------------------+------------------+
|  count|            25000|             25000|             25000|
|   mean|          12500.5| 67.99311359679979|127.07942116079916|
| stddev|7217.022700994273|1.9016787712056056|11.660897563604271|
|    min|                1|          60.27836|          78.01476|
|    max|            25000|           75.1528|           170.924|
+-------+-----------------+------------------+------------------+



df = [Index: int, Height: double ... 1 more field]


[Index: int, Height: double ... 1 more field]

In [6]:
println("Perform some SQL over CSV contents")
df.createOrReplaceTempView("people")
val df2 = spark.sql("SELECT * FROM people WHERE Height BETWEEN 68 AND 71")
df2.describe().show()

Perform some SQL over CSV contents
+-------+------------------+------------------+------------------+
|summary|             Index|            Height|            Weight|
+-------+------------------+------------------+------------------+
|  count|             11050|             11050|             11050|
|   mean|12598.144977375565| 69.22068299909508| 130.8976952787331|
| stddev| 7230.438697100867|0.8064637962888647|10.368627688958973|
|    min|                 3|          68.00003|          90.53995|
|    max|             25000|          70.99707|           170.924|
+-------+------------------+------------------+------------------+



df2 = [Index: int, Height: double ... 1 more field]


[Index: int, Height: double ... 1 more field]

In [7]:
println("Save CSV using Dataframe")
df2.repartition(5).write
   .format("com.databricks.spark.csv")
   .option("header", "true")
   .mode("overwrite")
   .save(hdfsPrefix + "/test/people-result.csv")

Save CSV using Dataframe


### Custom Maven dependency

In [8]:
print("Add Vegas lib")
%AddDeps org.vegas-viz vegas_2.11 0.3.11 --transitive
%AddDeps org.vegas-viz vegas-spark_2.11 0.3.11

Add Vegas libMarking org.vegas-viz:vegas_2.11:0.3.11 for download
Obtained 42 files
Marking org.vegas-viz:vegas-spark_2.11:0.3.11 for download
Obtained 2 files


### Show some graphs (finally!)

In [9]:
import vegas._
import vegas.sparkExt._

Vegas("Person Heights", width=400, height=100)
  .withDataFrame(df)
  .encodeX("Index", Quantitative)
  .encodeY("Height", Quantitative)
  .mark(Bar)
  .show

Vegas("Person Weights", width=400, height=100)
  .withDataFrame(df)
  .encodeX("Index", Quantitative)
  .encodeY("Weight", Quantitative)
  .mark(Bar)
  .show

<iframe id="frame-vegas-b63aaf14-7bee-4c41-82fc-9338a8a6ba8a" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-b63aaf14-7bee-4c41-82fc-9338a8a6ba8a'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 400.0,
 "height" : 100.0,
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "Index",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "Height",
 "type" : "quantitative"
 }
 },
 "description" : "Person Heights",
 "data" : {
 "values" : [
 {
 "Index" : 2,
 "Height" : 71.51521,
 "Weight" : 136.4873
 },
 {
 "Index" : 3,
 "Height" : 69.39874,
 "Weight" : 153.0269
 },
 {
 "Index" : 4,
 "Height" : 68.2166,
 "Weight" : 142.3354
 },
 {
 "Index" : 5,
 "Height" : 67.78781,
 "Weight" : 144.2971
 },
 {
 "Index" : 7,
 "Height" : 69.80204,
 "Weight" : 141.4947
 },
 {
 "Index" : 10,
 "Height" : 66.78236,
 "Weight" : 120.6672
 },
 {
 "Index" : 12,
 "Height" : 67.62333,
 "Weight" : 114.143
 },
 {
 "Index" : 13,
 "Height" : 68.30248,
 "Weight" : 125.6107
 },
 {
 "Index" : 14,
 "Height" : 67.11656,
 "Weight" : 122.4618
 },
 {
 "Index" : 15,
 "Height" : 68.27967,
 "Weight" : 116.0866
 },
 {
 "Index" : 25,
 "Height" : 67.62804,
 "Weight" : 141.8501
 },
 {
 "Index" : 32,
 "Height" : 65.8132,
 "Weight" : 120.7536
 },
 {
 "Index" : 33,
 "Height" : 67.8163,
 "Weight" : 125.7886
 },
 {
 "Index" : 35,
 "Height" : 71.80484,
 "Weight" : 140.1015
 },
 {
 "Index" : 37,
 "Height" : 66.80368,
 "Weight" : 141.7994
 },
 {
 "Index" : 38,
 "Height" : 67.65893,
 "Weight" : 121.2319
 },
 {
 "Index" : 39,
 "Height" : 67.80701,
 "Weight" : 131.3478
 },
 {
 "Index" : 45,
 "Height" : 65.98088,
 "Weight" : 106.4499
 },
 {
 "Index" : 46,
 "Height" : 68.67249,
 "Weight" : 128.7639
 },
 {
 "Index" : 47,
 "Height" : 66.88088,
 "Weight" : 145.6837
 },
 {
 "Index" : 51,
 "Height" : 69.91479,
 "Weight" : 147.0219
 },
 {
 "Index" : 52,
 "Height" : 67.33182,
 "Weight" : 126.3285
 },
 {
 "Index" : 54,
 "Height" : 69.10344,
 "Weight" : 115.7084
 },
 {
 "Index" : 57,
 "Height" : 70.40617,
 "Weight" : 155.8987
 },
 {
 "Index" : 58,
 "Height" : 66.54376,
 "Weight" : 128.0742
 },
 {
 "Index" : 59,
 "Height" : 66.36418,
 "Weight" : 119.3701
 },
 {
 "Index" : 60,
 "Height" : 67.537,
 "Weight" : 133.8148
 },
 {
 "Index" : 61,
 "Height" : 66.50418,
 "Weight" : 128.7325
 },
 {
 "Index" : 62,
 "Height" : 68.99958,
 "Weight" : 137.5453
 },
 {
 "Index" : 65,
 "Height" : 70.80592,
 "Weight" : 135.3165
 },
 {
 "Index" : 67,
 "Height" : 69.05914,
 "Weight" : 142.4684
 },
 {
 "Index" : 68,
 "Height" : 67.73103,
 "Weight" : 132.749
 },
 {
 "Index" : 76,
 "Height" : 66.35708,
 "Weight" : 120.2991
 },
 {
 "Index" : 77,
 "Height" : 68.36275,
 "Weight" : 138.6036
 },
 {
 "Index" : 78,
 "Height" : 65.4769,
 "Weight" : 132.9574
 },
 {
 "Index" : 80,
 "Height" : 67.72554,
 "Weight" : 122.524
 },
 {
 "Index" : 82,
 "Height" : 66.78405,
 "Weight" : 121.8986
 },
 {
 "Index" : 84,
 "Height" : 66.27848,
 "Weight" : 128.9418
 },
 {
 "Index" : 87,
 "Height" : 67.36436,
 "Weight" : 140.8901
 },
 {
 "Index" : 88,
 "Height" : 70.09297,
 "Weight" : 131.5916
 },
 {
 "Index" : 89,
 "Height" : 70.1766,
 "Weight" : 121.1232
 },
 {
 "Index" : 91,
 "Height" : 68.12932,
 "Weight" : 136.5479
 },
 {
 "Index" : 93,
 "Height" : 71.48752,
 "Weight" : 140.6104
 },
 {
 "Index" : 101,
 "Height" : 64.87434,
 "Weight" : 102.0927
 },
 {
 "Index" : 103,
 "Height" : 68.34761,
 "Weight" : 134.1842
 },
 {
 "Index" : 108,
 "Height" : 66.3146,
 "Weight" : 126.4772
 },
 {
 "Index" : 109,
 "Height" 

<iframe id="frame-vegas-c36e5f32-43e6-4363-bf6d-21d7982d891e" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-c36e5f32-43e6-4363-bf6d-21d7982d891e'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 400.0,
 "height" : 100.0,
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "Index",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "Weight",
 "type" : "quantitative"
 }
 },
 "description" : "Person Weights",
 "data" : {
 "values" : [
 {
 "Index" : 2,
 "Height" : 71.51521,
 "Weight" : 136.4873
 },
 {
 "Index" : 3,
 "Height" : 69.39874,
 "Weight" : 153.0269
 },
 {
 "Index" : 6,
 "Height" : 68.69784,
 "Weight" : 123.3024
 },
 {
 "Index" : 7,
 "Height" : 69.80204,
 "Weight" : 141.4947
 },
 {
 "Index" : 10,
 "Height" : 66.78236,
 "Weight" : 120.6672
 },
 {
 "Index" : 14,
 "Height" : 67.11656,
 "Weight" : 122.4618
 },
 {
 "Index" : 16,
 "Height" : 71.0916,
 "Weight" : 139.9975
 },
 {
 "Index" : 20,
 "Height" : 67.13118,
 "Weight" : 124.0449
 },
 {
 "Index" : 27,
 "Height" : 70.84235,
 "Weight" : 142.4235
 },
 {
 "Index" : 28,
 "Height" : 67.49434,
 "Weight" : 131.5502
 },
 {
 "Index" : 31,
 "Height" : 69.5233,
 "Weight" : 103.3016
 },
 {
 "Index" : 32,
 "Height" : 65.8132,
 "Weight" : 120.7536
 },
 {
 "Index" : 34,
 "Height" : 70.59505,
 "Weight" : 136.2225
 },
 {
 "Index" : 35,
 "Height" : 71.80484,
 "Weight" : 140.1015
 },
 {
 "Index" : 36,
 "Height" : 69.20613,
 "Weight" : 128.7487
 },
 {
 "Index" : 38,
 "Height" : 67.65893,
 "Weight" : 121.2319
 },
 {
 "Index" : 39,
 "Height" : 67.80701,
 "Weight" : 131.3478
 },
 {
 "Index" : 40,
 "Height" : 64.04535,
 "Weight" : 106.7115
 },
 {
 "Index" : 41,
 "Height" : 68.57463,
 "Weight" : 124.3598
 },
 {
 "Index" : 43,
 "Height" : 69.65814,
 "Weight" : 139.6711
 },
 {
 "Index" : 44,
 "Height" : 67.96731,
 "Weight" : 137.3696
 },
 {
 "Index" : 45,
 "Height" : 65.98088,
 "Weight" : 106.4499
 },
 {
 "Index" : 48,
 "Height" : 67.69868,
 "Weight" : 116.819
 },
 {
 "Index" : 50,
 "Height" : 69.08817,
 "Weight" : 134.9325
 },
 {
 "Index" : 51,
 "Height" : 69.91479,
 "Weight" : 147.0219
 },
 {
 "Index" : 53,
 "Height" : 70.26939,
 "Weight" : 125.4839
 },
 {
 "Index" : 54,
 "Height" : 69.10344,
 "Weight" : 115.7084
 },
 {
 "Index" : 56,
 "Height" : 70.18447,
 "Weight" : 147.8926
 },
 {
 "Index" : 57,
 "Height" : 70.40617,
 "Weight" : 155.8987
 },
 {
 "Index" : 58,
 "Height" : 66.54376,
 "Weight" : 128.0742
 },
 {
 "Index" : 68,
 "Height" : 67.73103,
 "Weight" : 132.749
 },
 {
 "Index" : 69,
 "Height" : 67.21568,
 "Weight" : 103.5275
 },
 {
 "Index" : 74,
 "Height" : 64.28508,
 "Weight" : 102.8351
 },
 {
 "Index" : 75,
 "Height" : 68.2452,
 "Weight" : 128.5214
 },
 {
 "Index" : 76,
 "Height" : 66.35708,
 "Weight" : 120.2991
 },
 {
 "Index" : 80,
 "Height" : 67.72554,
 "Weight" : 122.524
 },
 {
 "Index" : 82,
 "Height" : 66.78405,
 "Weight" : 121.8986
 },
 {
 "Index" : 83,
 "Height" : 70.05147,
 "Weight" : 155.3767
 },
 {
 "Index" : 87,
 "Height" : 67.36436,
 "Weight" : 140.8901
 },
 {
 "Index" : 91,
 "Height" : 68.12932,
 "Weight" : 136.5479
 },
 {
 "Index" : 92,
 "Height" : 70.24256,
 "Weight" : 141.4896
 },
 {
 "Index" : 94,
 "Height" : 69.20477,
 "Weight" : 112.1413
 },
 {
 "Index" : 95,
 "Height" : 70.06306,
 "Weight" : 133.457
 },
 {
 "Index" : 100,
 "Height" : 68.88741,
 "Weight" : 115.4759
 },
 {
 "Index" : 101,
 "Height" : 64.87434,
 "Weight" : 102.0927
 },
 {
 "Index" : 103,
 "Height" : 68.34761,
 "Weight" : 134.1842
 },
 {
 "Index" : 104,
 "Heig

### Stop application

In [10]:
println("Stop Spark session")
spark.stop()
//if you look in Spark Master UI, no application will be running after stop

Stop Spark session
